Họ tên: Vũ Minh Phát

MSSV: 21127739

# HW1: Giới thiệu CUDA

Với các GPU tương đối mới thì để biên dịch chỉ cần dùng câu lệnh: \
`nvcc tên-file.cu -o tên-file-chạy`

Nhưng trên Colab mình thường lấy được GPU khá cũ là Tesla K80 với compute capability (phiên bản phần cứng) là 3.7; để biên dịch đúng với GPU khá cũ này thì bạn cần dùng câu lệnh: \
`nvcc -arch=sm_37 tên-file.cu -o tên-file-chạy` \
Trong đó, 37 chính là compute capability của GPU Tesla K80.

Để phòng trường hợp khi làm bài bạn lấy được GPU có compute capability x.x nhưng khi chấm bài Thầy lại lấy được GPU có compute capability khác x.x, dưới đây mình sẽ có đoạn code Python để tự động lấy 2 con số ứng với compute capability của GPU và lưu vào 2 biến `major` và `minor`:


In [1]:
from numba import cuda
major, minor = cuda.get_current_device().compute_capability
print(f'GPU compute capability: {major}.{minor}')

GPU compute capability: 7.5


Một khi đã chạy đoạn code Python ở trên, để biên dịch thì bạn sẽ dùng câu lệnh: \
`nvcc -arch=sm_{major}{minor} tên-file.cu -o tên-file-chạy`

Dưới đây, khi làm bài thì bạn có thể tùy ý thêm/xóa cell. Đừng xóa mấy cell có chữ của Thầy là được.

## Câu 1

**Đề bài**: Viết chương trình chuyển ảnh RGB (ảnh màu) sang ảnh grayscale (ảnh xám) theo công thức:
<center>
giá-trị-grayscale = 0.299<span>&#215;</span>giá-trị-red + 0.587<span>&#215;</span>giá-trị-green + 0.114<span>&#215;</span>giá-trị-blue
</center>

`1.` Biên dịch chương trình

In [2]:
!nvcc -arch=sm_{major}{minor} HW1_P1.cu -o HW1_P1

`2.` Thử nghiệm chương trình với kích thước block tùy ý để đảm bảo mọi chuyện diễn ra đúng như mong đợi

In [3]:
!./HW1_P1 in.pnm out.pnm 32 16

Image size (width x height): 512 x 512

Processing time (use host): 3.010560 ms

GPU name: Tesla T4
GPU compute capability: 7.5
Processing time (use device): 3.096192 ms

Error between device result and host result: 0.002674


`3.` Chạy chương trình với các kích thước block khác nhau: 16x16, 32x32, 64x64

- Với kích thước block 16x16:

In [4]:
!./HW1_P1 in.pnm out.pnm 16 16

Image size (width x height): 512 x 512

Processing time (use host): 3.014688 ms

GPU name: Tesla T4
GPU compute capability: 7.5
Processing time (use device): 1.123904 ms

Error between device result and host result: 0.002674


- Với kích thước block 32x32:

In [5]:
!./HW1_P1 in.pnm out.pnm 32 32

Image size (width x height): 512 x 512

Processing time (use host): 3.406784 ms

GPU name: Tesla T4
GPU compute capability: 7.5
Processing time (use device): 1.103648 ms

Error between device result and host result: 0.002674


- Với kích thước block 64x64:

In [6]:
!./HW1_P1 in.pnm out.pnm 64 64

Image size (width x height): 512 x 512

Processing time (use host): 1.864448 ms

GPU name: Tesla T4
GPU compute capability: 7.5
Error: HW1_P1.cu:201, code: 9, reason: invalid configuration argument


**Nhận xét**:

`1.` **Về hiệu suất xử lý**:
- Khi kích thước block tăng từ 16x16 lên 32x32, thời gian xử lý có xu hướng giảm xuống, cho thấy khả năng tăng hiệu suất khi sử dụng các block lớn hơn. Điều này có thể là do block lớn hơn giúp tận dụng tốt hơn các tài nguyên tính toán của GPU, giảm thiểu số lượng block và do đó giảm overhead quản lý.

- Với block 64x64, thời gian xử lý của device không được đo vì lỗi cấu hình xuất hiện. Nguyên nhân có thể là do block 64x64 vượt quá giới hạn tài nguyên của GPU, dẫn đến lỗi cấu hình.

`2.` **Về sự khác biệt giữa kết quả của CPU và GPU**:
- Giá trị khác biệt trung bình giữa kết quả từ CPU và GPU rất nhỏ (khoảng 0.002674 trong cả hai trường hợp 16x16 và 32x32), cho thấy kết quả từ device gần như trùng khớp với kết quả từ host.

`3.` **Về khả năng xử lý lỗi**:
- Với kích thước block 64x64, chương trình đã phát hiện và báo lỗi đúng như mong đợi. Điều này cho thấy khả năng xử lý lỗi khi cấu hình vượt quá giới hạn tài nguyên, giúp tránh tình trạng crash hoặc lỗi không rõ nguyên nhân khi chạy với block lớn.

> **References**:  
> [1] Blog "How to Query Device Properties and Handle Errors in CUDA C/C++", section "Handling CUDA Errors", by Mark Harris - [developer.nvidia.com](https://developer.nvidia.com/blog/how-query-device-properties-and-handle-errors-cuda-cc/).  
> [2] GitHub repository "CUDA-RGB-grey" by Mohammadhossein Zarei - [github.com](https://github.com/mhezarei/CUDA-RGB-grey).  

## Câu 2

**Đề bài**: Viết chương trình làm mờ ảnh RGB. Để làm mờ ảnh RGB, ta sẽ thực hiện phép tích chập (convolution) giữa một filter (bộ lọc) với từng kênh màu của ảnh.

`1.` Biên dịch chương trình

In [7]:
!nvcc -arch=sm_{major}{minor} HW1_P2.cu -o HW1_P2

`2.` Thử nghiệm chương trình với kích thước block tùy ý để đảm bảo mọi chuyện diễn ra đúng như mong đợi

In [8]:
!./HW1_P2 in.pnm out2.pnm out2_target.pnm 32 16

Image size (width x height): 512 x 512

Processing time (use host): 419.608765 ms
Error: 0.000000

GPU name: Tesla T4
GPU compute capability: 7.5
Processing time (use device): 1.925696 ms
Error: 0.000703



`3.` Chạy chương trình với các kích thước block khác nhau: 16x16, 32x32, 64x64

- Với kích thước block 16x16:

In [9]:
!./HW1_P2 in.pnm out2.pnm out2_target.pnm 16 16

Image size (width x height): 512 x 512

Processing time (use host): 422.342102 ms
Error: 0.000000

GPU name: Tesla T4
GPU compute capability: 7.5
Processing time (use device): 2.032320 ms
Error: 0.000703



- Với kích thước block 32x32:

In [10]:
!./HW1_P2 in.pnm out2.pnm out2_target.pnm 32 32

Image size (width x height): 512 x 512

Processing time (use host): 420.653564 ms
Error: 0.000000

GPU name: Tesla T4
GPU compute capability: 7.5
Processing time (use device): 1.859712 ms
Error: 0.000703



- Với kích thước block 64x64:

In [11]:
!./HW1_P2 in.pnm out2.pnm out2_target.pnm 64 64

Image size (width x height): 512 x 512

Processing time (use host): 436.203156 ms
Error: 0.000000

GPU name: Tesla T4
GPU compute capability: 7.5
Error: HW1_P2.cu:248, code: 9, reason: invalid configuration argument


**Nhận xét**:

`1.` **Hiệu suất xử lý**:
- Nhìn chung, khi sử dụng CUDA, thời gian xử lý của GPU giảm đáng kể so với CPU (từ hơn 400 ms của host xuống khoảng 2 ms khi dùng device), cho thấy CUDA tận dụng tốt năng lực xử lý song song của GPU để làm mờ ảnh.

- Với các kích thước block khác nhau, thời gian xử lý trên device giảm nhẹ khi kích thước block tăng từ 16x16 lên 32x32, nhờ vào việc tận dụng tài nguyên hiệu quả hơn.

- Tuy nhiên, khi kích thước block tăng lên 64x64, chương trình không thể chạy được do lỗi cấu hình "invalid configuration argument". Điều này có thể là do kích thước block vượt quá giới hạn tài nguyên của GPU.

`2.` **Độ chính xác của kết quả**:
- Giá trị khác biệt trung bình giữa ảnh kết quả của device và ảnh kết quả đúng là rất nhỏ (khoảng 0.000703) cho cả hai trường hợp 16x16 và 32x32, cho thấy hàm kernel đã hoạt động đúng như mong đợi.

- Dường như không có lỗi giữa kết quả của host và ảnh chuẩn, chứng tỏ kết quả từ CPU là đáng tin cậy.

`3.` **Xử lý lỗi**:
- Với kích thước block 64x64, chương trình đã phát hiện và báo lỗi đúng như mong đợi. Điều này cho thấy khả năng xử lý lỗi khi cấu hình vượt quá giới hạn tài nguyên, giúp tránh tình trạng crash hoặc lỗi không rõ nguyên nhân khi chạy với block lớn.

> **References**:  
> [1] Question "What is the canonical way to check for errors using the CUDA runtime API?" - [stackoverflow.com](https://stackoverflow.com/questions/14038589/what-is-the-canonical-way-to-check-for-errors-using-the-cuda-runtime-api).  
> [2] GitHub repository "cuda-blur" by Ji Yan - [github.com](https://github.com/yanji84/cuda-blur).  